In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

ontology_url = 'gdrive/My Drive/Colab Notebooks/OntologiesRepresentationKnowledge/tanks_ontology.csv'

Mounted at /content/gdrive


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [3]:
df = pd.read_csv(ontology_url)

In [4]:
df.isna().sum()

subject        0
predicate      0
object       445
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df

,subject,predicate,object
0,player_2685498_stats_on_15697,concreteTankPiercings,72
1,player_4991449_stats_on_3681,concreteTankAvgDamageBlocked,686.84
2,player_12177534_stats_on_19969,concreteTankPiercings,1529
3,player_12050543_stats_on_2433,concreteTankDamageReceived,138189
4,player_14636191_stats_on_15697,concreteTankDirectHitsReceived,176
...,...,...,...
4595023,player_6106224_stats_on_13825,concreteTankLosses,251
4595024,player_20304852_stats_on_13089,concreteTankBattleAvgXp,499
4595025,player_3202392_stats_on_63537,concreteTankFrags,78
4595026,player_4865908_stats_on_2433,concreteTankMarkOfMastery,4


In [7]:
!pip install pykeen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.7/717.7 KB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 36.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.0/81.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.4/210.4 KB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.1/147.1 KB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [8]:
from pykeen.triples import TriplesFactory
from pykeen.pipeline import pipeline

tf = TriplesFactory.from_labeled_triples(
  df[["subject", "predicate", "object"]].values
)

training, testing, validation = tf.split([.8, .1, .1])

INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [3163263, 459458, 459459]


In [9]:
result = pipeline(
    training=training,
    testing=testing,
    validation=validation,
    model='RotatE',
    stopper='early',
    epochs=1,
    dimensions=512,
    random_seed=420
)

INFO:pykeen.pipeline.api:Using device: None
INFO:pykeen.stoppers.early_stopping:Inferred checkpoint path for best model weights: /root/.data/pykeen/checkpoints/best-model-weights-82cd3162-6350-4b21-bef1-7bdff37fa141.pt


Training epochs on cuda:0:   0%|          | 0/1 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/14359 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=8.


Evaluating on cuda:0:   0%|          | 0.00/459k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 10591.60s seconds


In [27]:
model = result.model
relation_prediction_df_1 = model.get_relation_prediction_df(head_label='player_IVIargo', tail_label='clan_101221', triples_factory=tf)
relation_prediction_df_1

,relation_id,relation_label,score,in_training
100,100,isMemberOfClan,-1.089079,True
105,105,leaderId,-1.198037,False
33,33,concreteTankHitsPercents,-1.210373,False
82,82,http://www.w3.org/2000/01/rdf-schema#comment,-1.228237,False
120,120,rapidSpeedBackward,-1.242410,False
...,...,...,...,...
26,26,concreteTankDirectHitsReceived,-1.350756,False
158,158,vechicleGunReloadTime,-1.352167,False
96,96,http://www.w3.org/2002/07/owl#oneOf,-1.353205,False
124,124,rapidSwitchOnTime,-1.353328,False


In [26]:
relation_prediction_df_2 = model.get_relation_prediction_df(head_label='player_SV608', tail_label='battle_14203412_results_of_SV608', triples_factory=tf)
relation_prediction_df_2

,relation_id,relation_label,score,in_training
89,89,http://www.w3.org/2002/07/owl#equivalentClass,-2.541450,False
79,79,http://www.w3.org/1999/02/22-rdf-syntax-ns#first,-2.585977,False
146,146,vechicleArmorHullSides,-2.593021,False
102,102,isResultsOfBattle,-2.596216,False
76,76,http://purl.org/dc/elements/1.1/description,-2.608458,False
...,...,...,...,...
30,30,concreteTankExplosionHitsReceived,-2.871676,False
18,18,concreteTankAccountId,-2.873617,False
85,85,http://www.w3.org/2000/01/rdf-schema#subClassOf,-2.895115,False
0,0,accountId,-2.922361,False
